In [1]:
!pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [2]:
!pip install torch torchvision torchaudio


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


In [4]:
!pip install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import torch
from datasets import load_from_disk
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DistilBertTokenizer
from transformers import EarlyStoppingCallback

In [7]:
# Loading the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Loading the tokenized datasets from local disk
token_train = load_from_disk('/content/drive/MyDrive/train_data')
token_test = load_from_disk('/content/drive/MyDrive/test_data')


In [9]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2, dropout=0.3)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install transformers[torch] accelerate

In [11]:
import os

# Ensuring that the 'results' and 'logs' directories exist
os.makedirs('/content/drive/MyDrive/results', exist_ok=True)
os.makedirs('/content/drive/MyDrive/logs', exist_ok=True)

In [12]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',          
    evaluation_strategy='epoch',                          
    learning_rate=2e-5,                                   
    per_device_train_batch_size=16,                       
    per_device_eval_batch_size=64,                        
    num_train_epochs=3,                                   
    weight_decay=0.1,                                     
    logging_dir='/content/drive/MyDrive/logs',            
    logging_steps=10                                      
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Initializing the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train,
    eval_dataset=token_test,
)

In [14]:
# Training the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.176200,0.204249
2,0.284900,0.227235
3,0.156200,0.251236


TrainOutput(global_step=18750, training_loss=0.18513279186407725, metrics={'train_runtime': 3974.1354, 'train_samples_per_second': 75.488, 'train_steps_per_second': 4.718, 'total_flos': 9935054899200000.0, 'train_loss': 0.18513279186407725, 'epoch': 3.0})

In [15]:
# Evaluate the model on the test set
results = trainer.evaluate()

print(results)

{'eval_loss': 0.2512359619140625, 'eval_runtime': 96.6136, 'eval_samples_per_second': 258.763, 'eval_steps_per_second': 4.047, 'epoch': 3.0}


In [16]:
import os

os.makedirs('/content/drive/MyDrive/distilbert_sentiment_model', exist_ok=True)

In [17]:
# Saving the fine-tuned model and tokenizer
model.save_pretrained('/content/drive/MyDrive/distilbert_sentiment_model')
tokenizer.save_pretrained('/content/drive/MyDrive/distilbert_sentiment_model')

('/content/drive/MyDrive/distilbert_sentiment_model/tokenizer_config.json',
 '/content/drive/MyDrive/distilbert_sentiment_model/special_tokens_map.json',
 '/content/drive/MyDrive/distilbert_sentiment_model/vocab.txt',
 '/content/drive/MyDrive/distilbert_sentiment_model/added_tokens.json')

In [18]:

model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.3, inplace=False)
 